In [ ]:
import sys

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as immg
from pathlib import Path
import os
import seaborn as sns
import gc
import torchvision
import cv2
from fastai.data.all import *
from fastai.vision.core import *
from fastai.vision.data import *
from tqdm.notebook import tqdm
import zipfile
import io
from sklearn.decomposition import PCA
from fastai.vision.all import *

## Function to Create Stratified K_fold for both Regression as well as Classification

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')

In [ ]:
trn_df = pd.read_csv('../input/cassav-cleaned-df/cleaned_df.csv')
trn_df.head()

In [ ]:
trn_df[trn_df.kfold==2].label.value_counts()

## Train and Valid idxs

In [ ]:
trn_idx,val_idx = trn_df[trn_df.kfold!=2].index, trn_df[trn_df.kfold==2].index

## Transforms

In [ ]:
item_tfms = RandomResizedCrop(256, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=256, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=64

## DataLoader

In [ ]:
def get_data(FOLD):
    trn_idx,val_idx = trn_df[trn_df.kfold!=FOLD].index, trn_df[trn_df.kfold==FOLD].index
    dls = ImageDataLoaders.from_df(trn_df, path, fn_col = 'image_id', folder='train_images',label_col='label',
                                  bs=32,y_block=CategoryBlock,item_tfms=item_tfms,batch_tfms=batch_tfms,val_idxs=val_idx)
    return dls

## Model

In [ ]:
!pip install -q efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
class MyModel(Module):
    def __init__(self, num_classes):

        self.effnet = EfficientNet.from_pretrained("efficientnet-b4")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1792, num_classes)

    def forward(self, image):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs

## Learner

In [ ]:
for fold in range(2):
    
    dls = get_data(fold)
    effnet = MyModel(dls.c)
    
    learn = Learner(dls, 
                effnet,
                loss_func = LabelSmoothingCrossEntropyFlat(), 
                metrics = [accuracy], 
                cbs=[MixUp()],
                model_dir='/kaggle/working/').to_native_fp16()
    
    nm = 'best_model_fold_'+str(fold)
    cb1 = SaveModelCallback(monitor='accuracy',fname=nm,comp=np.greater) # Callbacks
    cb2 = ReduceLROnPlateau(monitor='valid_loss', min_delta=0.1, patience=2,factor=0.2)
    learn.fit_one_cycle(10, 0.0007556, cbs = [cb1,cb2])
    learn.load(nm);
    learn = learn.to_fp32()
    name = 'best_model_fp32_'+'fold_'+str(fold)
    learn.save(name,with_opt=True);
    learn,dls=None,None
    gc.collect()